# Media

In [1]:
import datetime
print(f"Last updated: {str(datetime.datetime.now())[:10]}")

Last updated: 2025-12-12


Since around 2020, I have been keeping track of almost all media I consume - mostly books and games, but also concerts (but not CD's), movies, museums, and other forms of culture. Here, you can find a quick summary of what I've been up to

## Last enjoyed

In [2]:
#GNU STP
import pandas as pd
import math
df = pd.read_csv("media.csv")

verb_dict = {"Book":"read","Game":"played","Movie":"watched","Concert":"visited","Museum":"visited"}
end = f" by {df['creator'].iloc[-1]}" if not any(df['creator'].iloc[-1:].isna()) else ""
print(f"Last {verb_dict.get(df['type'].iloc[-1], 'visited')}: {df['name'].iloc[-1]}{end}")
score_dict = {10:'close to perfect!', 9:'fantastic!', 8:'great', 7:'good', 6:'okay', 5:'meh...', 4:'not great', 3:'bad', 2:'terrible', 1: 'abhorrent', 0:'the worst!'}
print(f"Which I thought was: {score_dict[math.floor(df['score'].iloc[-1])]}")

Last read: Hoofdstukken Vermogensrecht by Hans Nieuwenhuis
Which I thought was: not great


## Favorites

A good story, to me, is defined as something that keeps you thinking about long after you finished reading/watching/playing it. Hence, even if I didn't really enjoy something at the moment itself, it may turn out to be a favorite later. Here are my some of my favorite pieces of media for each of the years I have been keeping track:

<details>
  <summary>2024</summary>

  - _Wij Slaven van Suriname_ by Anton de Kom
  - _Live in AFAS_ by King Gizzard and the Lizard Wizard
  - _Tactical Breach Wizards_ by Tom Francis
  - _The Planets & Sinfonia Antarctica_ by Rotterdams Philharmonisch Orkest
</details>
<details>
  <summary>2023</summary>

  - _The Legend of Zelda: A Link Between Worlds_ by Nintendo
  - _Chain-Gang All-Stars_ by Nana Kwame Adjei-Brenyah
  - _Pinkpop 2023_
  - _Dr. Atomic_ by Utrechtsch Studenten Concert
  - _The MANIAC_ by Benjamín Labatut
</details>
<details>
  <summary>2022</summary>

  - _Ficciones_ by Jorge Luis Borges
  - _Ontjoodst door de Wetenschap_ by Hans Jessurun d'Oliveira (ed.)
  - _Educated_ by Tara Westover
  - _Sable_ by Shedworks
</details>
<details>
  <summary>2021</summary>

  - _The City and the City_ by China Miéville
  - _Sekiro: Shadows Die Twice_ by FromSoftware
  - _Stories of Your Life and Others_ and _Exhalation_ by Ted Chiang
  - _Wat Wij Zagen_ by Hanna Bervoets
  - _When We Cease to Understand the World_ by Benjamín Labatut
  - _When Einstein Walked with Gödel_ by Jim Holt
</details>

## What?

In [3]:
import plotly.express as px
def widthsplitter(s, width):
    s_split = s.split(' ')
    output = []
    current = ""
    for item in s_split:
        current += item + ' '
        if len(current) > width:
            output.append(current)
            current = ""
    if current != "":
        output.append(current)
    return output

def rounder(score):
    return float(round(score*2))/2

def customwrap(s, width=10):
    return "<br>".join(widthsplitter(s, width))

df4 = df.copy()
df4[['year','month']] = df4['date'].str.split('-',expand=True)
df4['scoreactual'] = df4['score'].apply(lambda x: rounder(x))
df4['displayname'] = [
    customwrap(f"{name} by {creator}") if not pd.isna(creator) else customwrap(f"{name}")
    for name, creator in zip(df4['name'], df4['creator'])
    ]
fig = px.sunburst(
    df4, path=['type', 'year', 'scoreactual', 'displayname'], maxdepth=2,
)
fig.update_layout(margin={'t':5,'l':5,'b':5,'r':5}, showlegend=False)
fig.update_traces(sort=False, hovertemplate='<b>Amount: %{value} </b><br>%{label}')
fig.show()

<p style="text-align: center;">↑ Click me!</p>

## How much?

In [4]:
# Split year and month values
df2 = df.copy()
df2[['year','month']] = df2['date'].str.split('-',expand=True)
df2 = df2.drop(columns=['date'])

# All possible values for types, years and months
types = ["All", "Game", "Book", "Other"]
years = [str(x) for x in range(min(df2['year'].astype(int)), max(df2['year'].astype(int)) + 1)]
months = [f"{month:>2}".replace(' ', '0') for month in range(1, 13)]

# Create a default where all values are zero
idx = pd.MultiIndex.from_product([years, months, types], names=['year', 'month', 'type'])
base = pd.DataFrame(index=idx).reset_index()
base['count'] = [0.0] * len(base)

# Calculate actual values and sum of values with groupby 
df2.loc[(df2['type'] != "Game") & (df2['type'] != "Book"), ['type']] = "Other"
df2 = df2[['year', 'month', 'type', 'name']].groupby(['year', 'month', 'type']).count().reset_index()
df2 = df2.rename(columns={"name": "count"})
all = df2[['year', 'month', 'count']].groupby(['year', 'month']).sum().reset_index()
all['type'] = ['All'] * len(all)
df2 = pd.concat([df2, all])

# Merge default values and calculated values and sum these
df2 = base.merge(df2, how='outer')
df2 = df2.groupby(['year', 'month', 'type'])['count'].sum().reset_index()

# Calculate mean values
means = df2[df2['year'] != str(max(years))].groupby(['month', 'type'])['count'].mean().reset_index()
means['year'] = ['Mean'] * len(means)
df2 = pd.concat([df2, means])

# Create plot
fig = px.line(df2, x='month', y='count', color='year', animation_frame='type',
                color_discrete_map={"Mean": 'black'}, custom_data=['year'], template='plotly_white',
                labels={"month": "Month", "count": "Amount", "year": "Year"},
                hover_data={'type':False, 'month':False, 'count':False})
fig.update_layout(margin={'t':10,'l':0,'b':0,'r':10}, yaxis_range=[0,15], xaxis_range=[0,11],
                    xaxis = dict(
                        tickmode = 'array',
                        tickvals = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],
                        ticktext = ['', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', '']
                    ),
                    legend = dict(orientation="h", yanchor='bottom', y=0, xanchor='right', x=1)
                )
fig.update_traces(opacity=.4)
fig.update_traces(connectgaps=False)
fig.update_traces(selector=dict(line_color="black"), opacity=1)
fig.update_traces(hovertemplate='%{customdata[0]}')
fig['layout']['sliders'][0]['pad']=dict(r=100, t= 20, b=20)
fig["layout"].pop("updatemenus")
fig.show()


## Albums
One particular interest I have is collecting albums out of Rolling Stone's Top 500 Albums list. Below, you can see how many I already have (out of the 2020 list).

In [5]:
adf = pd.read_csv("albums.csv", sep=';')
adf['Rounded year'] = adf['Year'].apply(lambda x: 10 * round(int(x) / 10))
adf['Have'] = adf['Have'].apply(lambda x: 1 if x == "X" else x)

labels = []
percentages = []
got_arr = []; need_arr = []
    
for year in ["Total"] + sorted(adf['Rounded year'].unique()):
    if year == "Total": 
        need = len(adf); got = len(adf[(adf['Have'] == 1)])
    else: 
        need = len(adf[adf['Rounded year'] == year])
        got = len(adf[(adf['Rounded year'] == year) & (adf['Have'] == 1)])
    score = got/need
    got_arr.append(got); need_arr.append(need)
    percentages.append((got/need) * 100)
    labels.append(f"{year-5}-{year+4}" if year != "Total" else "Total")

colors = ["complete_color" if perc == 100 else "incomplete_color" for perc in percentages]
colors[0] = "total_color" if colors[0] == "incomplete_color" else "complete_color"

customdata = [got_arr, need_arr]

d = {"Decades": labels, "Progress": percentages}
df1 = pd.DataFrame(data=d)
fig = px.bar(df1, x="Decades", y="Progress", template='plotly_white', color=colors, custom_data=customdata,
             color_discrete_map={"total_color":"#ef553b", "complete_color":"#FECB52", "incomplete_color":"#636efa"})
fig.update_layout(yaxis_range=[0,100], showlegend=False)
fig.update_traces(hovertemplate='%{x} progress:<br>%{customdata[0]} out of %{customdata[1]}<extra></extra>')
fig.update_xaxes(tickangle=90)
fig.show()